In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image,ImageOps
import os

Using TensorFlow backend.


In [2]:
INPUT_IMAGE = './input/input.jpg'

EMNIST_PATH = './data/emnist/'
mapping_processed = EMNIST_PATH + 'processed-mapping.csv'

In [3]:
def img2emnist(filepath, char_code):
    img = Image.open(filepath).resize((28, 28))
    inv_img = ImageOps.invert(img)
    
    flatten = np.array(inv_img).flatten()
    flatten = flatten / 255
    flatten = np.where(flatten > 0.5, 1, 0)
    
    csv_img = ','.join([str(num) for num in flatten])
    csv_str = '{},{}'.format(char_code, csv_img)
    return csv_str

In [4]:
model = load_model('model/alphanum_model_binary_ex_88.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [5]:
temp_filename = 'test.csv'
if os.path.exists(temp_filename):
    os.remove(temp_filename)

f_test = open(temp_filename, 'a+')
files = [f for r, d, f in os.walk('segmented')][0]
for f in files:
    csv = img2emnist('segmented/' + f, -1)
    print(csv, file=f_test)
    
f_test.close()

In [6]:
names = ["label"] + ["pixel" + str(i) for i in range(784)]
                     
data = pd.read_csv(temp_filename, header=None, names=names)
X_data = data.drop(labels = ["label"], axis = 1)
X_data = X_data.values.reshape(-1,28,28,1)
del data

In [7]:
df = pd.read_csv(mapping_processed)
code2char = {}
for index, row in df.iterrows():
    code2char[row['id']] = row['char']

In [8]:
# predict results
results = model.predict(X_data)

# select the index with the maximum probability
results = np.argmax(results, axis = 1)
parsed_str = ""
for r in results:
    parsed_str += code2char[r]

parsed_str

'X4tQXt4UD'